## Step 1: Loading Data

In [33]:
# Importing modules
import pandas as pd


# Read data into papers
docs = pd.read_csv('Pubmed5k.csv')

# Print head
docs.head()

,ArticleID,Title,Abstract
0,34153941,Stable Coordination Variability in Overground ...,Coordination variability (CV) is commonly anal...
1,34153942,Weak Hip Strength Increases Dynamic Knee Valgu...,Clinical Scenario: Dynamic knee valgus (DKV) i...
2,34153964,Current and Future Projections of Amyotrophic ...,Various methodologies have been reported to as...
3,34153968,Disparities between Asian and Non-Asian Thromb...,As outcomes for acute ischemic stroke (AIS) va...
4,34153978,Maternal Factors Predicting Loss to Follow-Up ...,Because hearing loss in children can result in...


In [39]:
with pd.option_context('display.max_colwidth', None):
  display(docs)

,title,abstract
0,Stable Coordination Variability in Overground Walking and Running at Preferred and Fixed Speeds.,"Coordination variability (CV) is commonly analyzed to understand dynamical qualities of human locomotion. The purpose of this study was to develop guidelines for the number of trials required to inform the calculation of a stable mean lower limb CV during overground locomotion. Three-dimensional lower limb kinematics were captured for 10 recreational runners performing 20 trials each of preferred and fixed speed walking and running. Stance phase CV was calculated for 9 segment and joint couplings using a modified vector coding technique. The number of trials required to achieve a CV mean within 10% of 20 strides average was determined for each coupling and individual. The statistical outputs of mode (walking vs running) and speed (preferred vs fixed) were compared when informed by differing numbers of trials. A minimum of 11 trials were required for stable mean stance phase CV. With fewer than 11 trials, CV was underestimated and led to an oversight of significant differences between mode and speed. Future overground locomotion CV research in healthy populations using a vector coding approach should use 11 trials as a standard minimum. Researchers should be aware of the notable consequences of an insufficient number of trials for overall study findings."
1,Weak Hip Strength Increases Dynamic Knee Valgus in Single-Leg Tasks of Collegiate Female Athletes.,"Clinical Scenario: Dynamic knee valgus (DKV) is a mechanical alteration in the knee that leads to increased risk of injury. Weakness of hip musculature in hip abduction (HABD), extension (HEXT), and external rotation (HER) may contribute to increased DKV in single-leg landing tasks. Focused Clinical Question: Is decreased hip strength associated with an increase in DKV during a single-leg landing task in collegiate female athletes? Summary of Key Findings: Three studies were included: One randomized control trial (RCT), one cohort study, and one case-control. All three studies found that decreases in HABD and HER strength contributed to increased DKV during single-leg landing tasks. One study also found that the hip extensors contribute to controlling hip adduction, a common factor in many mechanisms of injuries. These three studies recommended strengthening HABD, HEXT, and HER to decrease DKV and reduce the risk of injury at the knee. Clinical Bottom Line: Weak HABD, HEXT, and HER contribute to increased DKV in college female athletes, but strengthening HABD, HEXT, and HER can lead to decreases in DKV and, overall, reduce the risk of injury at the knee. Strength of Recommendation: These articles were graded with a level of evidence of III or higher, giving a grade of B strength of recommendation that weak HABD, HEXT, and HER are associated with increased DKV in collegiate female athletes."
2,Current and Future Projections of Amyotrophic Lateral Sclerosis in the United States Using Administrative Claims Data.,"Various methodologies have been reported to assess the real-world epidemiology of amyotrophic lateral sclerosis (ALS) in the United States. The aim of this study was to estimate the prevalence, incidence, and geographical distribution of ALS using administrative claims data and to model future trends in ALS epidemiology.,We performed a retrospective analysis of deidentified administrative claims data for >100 million patients, using 2 separate databases (IBM MarketScan Research Databases and Symphony Health Integrated DataVerse [IDV]), to identify patients with ALS. We evaluated disease prevalence, annual incidence, age- and population-controlled geographical distribution, and expected future trends.,From 2013 to 2017, we identified 7,316 and 35,208 ALS patients from the MarketScan databases and IDV, respectively. Average annual incidence estimates were 1.48 and 1.37 per 100,000 and point prevalence estimates were 6.85 and 5.16 per 100,000 and in the United

In [34]:
docs.shape

(4999, 3)


## Step 2: Data Cleaning

In [35]:
docs.columns= docs.columns.str.lower()
docs.head()

,articleid,title,abstract
0,34153941,Stable Coordination Variability in Overground ...,Coordination variability (CV) is commonly anal...
1,34153942,Weak Hip Strength Increases Dynamic Knee Valgu...,Clinical Scenario: Dynamic knee valgus (DKV) i...
2,34153964,Current and Future Projections of Amyotrophic ...,Various methodologies have been reported to as...
3,34153968,Disparities between Asian and Non-Asian Thromb...,As outcomes for acute ischemic stroke (AIS) va...
4,34153978,Maternal Factors Predicting Loss to Follow-Up ...,Because hearing loss in children can result in...


In [36]:
del docs["articleid"]
docs.head()

,title,abstract
0,Stable Coordination Variability in Overground ...,Coordination variability (CV) is commonly anal...
1,Weak Hip Strength Increases Dynamic Knee Valgu...,Clinical Scenario: Dynamic knee valgus (DKV) i...
2,Current and Future Projections of Amyotrophic ...,Various methodologies have been reported to as...
3,Disparities between Asian and Non-Asian Thromb...,As outcomes for acute ischemic stroke (AIS) va...
4,Maternal Factors Predicting Loss to Follow-Up ...,Because hearing loss in children can result in...


In [37]:
docs.shape

(4999, 2)

### Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of abstract column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [40]:
# Load the regular expression library
import re
# Remove punctuation
docs['abstract_processed'] = \
docs['abstract'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
docs['abstract_processed'] = \
docs['abstract_processed'].map(lambda x: x.lower())

# Print out the first rows of docs
docs['abstract_processed'].head()

0    coordination variability (cv) is commonly anal...
1    clinical scenario: dynamic knee valgus (dkv) i...
2    various methodologies have been reported to as...
3    as outcomes for acute ischemic stroke (ais) va...
4    because hearing loss in children can result in...
Name: abstract_processed, dtype: object

### Tokenize words and further clean-up text
Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [41]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = docs.abstract_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['coordination', 'variability', 'cv', 'is', 'commonly', 'analyzed', 'to', 'understand', 'dynamical', 'qualities', 'of', 'human', 'locomotion', 'the', 'purpose', 'of', 'this', 'study', 'was', 'to', 'develop', 'guidelines', 'for', 'the', 'number', 'of', 'trials', 'required', 'to', 'inform']


## Step 3: Phrase Modeling: Bigram and Trigram Models

In [42]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Remove Stopwords, Make Bigrams and Lemmatize
The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [45]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [47]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [49]:
!python -m spacy download en_core_web_sm

You should consider upgrading via the 'C:\Users\FreeComp\anaconda3\envs\data\python.exe -m pip install --upgrade pip' command.



[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [50]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['coordination', 'cv', 'commonly', 'analyze', 'understand', 'dynamical', 'quality', 'human', 'locomotion', 'purpose', 'study', 'develop', 'guideline', 'number', 'trial', 'require', 'inform', 'calculation', 'stable', 'mean', 'cv', 'overground', 'locomotion', 'kinematic', 'capture', 'recreational', 'runner', 'perform', 'trial', 'preferred']


## Step 4: Data transformation: Corpus and Dictionary
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. 

In [51]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 6), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


## Step 5: Base Model

In [52]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [56]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"cell" + 0.013*"protein" + 0.008*"expression" + 0.007*"use" + '
  '0.007*"gene" + 0.006*"study" + 0.006*"induce" + 0.005*"show" + '
  '0.005*"mechanism" + 0.005*"role"'),
 (1,
  '0.014*"patient" + 0.008*"study" + 0.008*"use" + 0.007*"treatment" + '
  '0.007*"bone" + 0.007*"follow" + 0.006*"case" + 0.006*"surgical" + '
  '0.005*"perform" + 0.005*"group"'),
 (2,
  '0.037*"patient" + 0.017*"group" + 0.012*"study" + 0.010*"treatment" + '
  '0.010*"high" + 0.009*"risk" + 0.008*"year" + 0.008*"age" + 0.008*"day" + '
  '0.008*"use"'),
 (3,
  '0.040*"specie" + 0.016*"new" + 0.011*"describe" + 0.007*"genus" + '
  '0.007*"study" + 0.006*"base" + 0.006*"include" + 0.006*"provide" + '
  '0.006*"region" + 0.005*"distribution"'),
 (4,
  '0.005*"drought" + 0.005*"chapter" + 0.004*"allergy" + 0.004*"epitope" + '
  '0.003*"formulation" + 0.003*"allergic" + 0.003*"fetus" + 0.003*"starch" + '
  '0.003*"plaque" + 0.003*"pot"'),
 (5,
  '0.016*"patient" + 0.012*"disease" + 0.011*"infection" + 

### Compute Model Perplexity and Coherence Score

In [57]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4148432180785776
